In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils import data
from torch import optim
from adabelief_pytorch import AdaBelief
from ACProp import ACProp
import torch.nn as nn
import torch.nn.functional as F

import numpy as np

from copy import deepcopy
from tqdm.notebook import tqdm
from itertools import product
import matplotlib.pyplot as plt

In [3]:
def read_npy(filepath):
    with open(filepath, 'rb') as file:
        return np.load(file)
    
def prepare_spectra(spectra, seed=2022):
    # normalization
    spectra = (spectra - spectra.min()) / (spectra.max() - spectra.min())
    # permutation
    np.random.seed(seed)
    idx = np.random.permutation(spectra.shape[0])
    spectra = spectra[idx]
    
    return spectra

def prepare_params(params):
    params = np.reshape(params, (params.shape[0], -1))
    return params
    
def split_data(X, y, val_size=0.2):
    idx_slice = int(X.shape[0] * val_size)
    X_train, X_test = X[idx_slice:], X[:idx_slice]
    y_train, y_test = y[idx_slice:], y[:idx_slice]
    return X_train, X_test, y_train, y_test

def make_loader(inputs, targets, batch_size=256):
    loader = data.DataLoader(list(zip(inputs, targets)), batch_size=batch_size, drop_last=False, shuffle=False)
    return loader

In [4]:
spectra = read_npy('spectrums.npy')
params = read_npy('parameters.npy')

spectra = prepare_spectra(spectra)
params = prepare_params(params)

X_train, X_test, y_train, y_test = split_data(spectra, params, 0.1)

train_loader = make_loader(X_train, y_train)
test_loader = make_loader(X_test, y_test)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(150, 100)
        self.fc2 = nn.Linear(100, 75)
        self.fc3 = nn.Linear(75, 2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## fitting

In [9]:
def fit_model(model, optimizer, loss_function, train_loader, test_loader, epochs=50):
    train_losses = []
    test_losses = []
    
    for epoch in (list(range(epochs))):
        train_loss = 0
        model.train()
        for idx, (inputs, targets) in enumerate(train_loader): 
            inputs, targets = inputs.float(), targets.float()
            outputs = model(inputs)
            loss = loss_function(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        train_losses.append((train_loss / (idx + 1)))

        test_loss = 0
        model.eval()
        with torch.no_grad():
            for idx, (inputs, targets) in enumerate(test_loader):
                inputs, targets = inputs.float(), targets.float()
                outputs = model(inputs)
                loss = loss_function(outputs, targets)
                test_loss += loss.item()
        test_losses.append(test_loss / (idx + 1))
        
    return train_losses, test_losses

In [14]:
learning_rate_grid = np.array([1e-6, 5e-6, 1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 1e-1, 1.0])

In [18]:
loss_function = nn.MSELoss()

train_losses_grid = []
test_losses_grid = []

optimizers = [
    AdaBelief(
        net.parameters(),
        lr=1e-4,
        eps=1e-16,
        betas=(0.9, 0.999),
        weight_decouple=False,
        rectify=False,
        print_change_log=False,
    ),
    ACProp(
        net.parameters(),
        lr=lr,
        eps=1e-8,
        betas=(0.9, 0.999),
        weight_decouple=True,
        weight_decay=1e-4,
        rectify=False,
    ),
]

for lr, optimizer in tqdm(zip(learning_rate_grid, optimizers)):
    net = Net()
    train_losses, test_losses = fit_model(
        model=net,
        optimizer=optimizer,
        loss_function=loss_function,
        train_loader=train_loader,
        test_loader=test_loader,
        epochs=50,
    )
    train_losses_grid.append(train_losses)
    test_losses_grid.append(test_losses)
    
    with open(f'test_losses_grid_{optimizer.__class__.__name__}.npy', 'wb') as file:
        np.save(file, np.array(test_losses_grid))
        
    with open(f'train_losses_grid_{optimizer.__class__.__name__}.npy', 'wb') as file:
        np.save(file, np.array(test_losses_grid))
        
with open(f'learning_rate_grid.npy', 'wb') as file:
    np.save(file, np.array(learning_rate_grid))

  0%|          | 0/12 [00:00<?, ?it/s]

Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
Weight decoupling enabled in AdaBelief
